In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2021-09-22_raw.csv to 2021-09-22_raw.csv


In [ ]:
uploaded_sec = files.upload()

Saving merge_clean_v2.csv to merge_clean_v2.csv


In [21]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import re
import networkx
import numpy as np
import ast
import nltk
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from wordcloud import WordCloud

In [22]:
df = pd.read_csv('/content/from_model_v2.csv', encoding="utf-8")
df

,Unnamed: 0,username,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,tweet,verdict,RT,source_user,target_user
0,0,PIACaraga,2021-09-21 23:53:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,"TINGNAN: Kahapon, Setyembre 20, inilabas ni @n...",TRUE,True,pcoogov,PIACaraga
1,1,PIACaraga,2021-09-21 23:53:27+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,PIACaraga
2,2,PIACaraga,2021-09-21 23:52:23+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,"PANOORIN: Kahapon, Setyembre 20, nagbigay ng u...",TRUE,True,pcoogov,PIACaraga
3,3,YamauchiElena,2021-09-21 20:45:11+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,YamauchiElena
4,4,greg_kassie,2021-09-21 16:28:08+00:00,,1.440000e+18,tl,0.0,0.0,0.0,34.0,Dapat mayroong sapat na abiso sa mga lugar na ...,MOSTLY_TRUE,True,WinGatchalian74,greg_kassie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4862,4862,roysant51162134,2021-09-17 08:29:20+00:00,,1.440000e+18,tl,0.0,0.0,1.0,0.0,@EdsonCGuido @ABSCBNNews habang dumadami nabab...,MOSTLY_FALSE,False,roysant51162134,ABSCBNNews
4863,4863,dzrhnews,2021-09-17 04:23:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,Pakiusap na payagan na rin ang mga naka-unang ...,TRUE,False,dzrhnews,liezel_once
4864,4864,GrowlHarhar,2021-09-17 03:09:24+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,@CityMouse14 @inquirerdotnet @KAguilarINQ Ikaw...,MOSTLY_TRUE,False,GrowlHarhar,inquirerdotnet
4865,4865,GrowlHarhar,2021-09-17 03:09:24+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,@CityMouse14 @inquirerdotnet @KAguilarINQ Ikaw...,MOSTLY_TRUE,False,GrowlHarhar,KAguilarINQ


In [23]:
df.created_at = pd.to_datetime(df['created_at'])
df

,Unnamed: 0,username,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,tweet,verdict,RT,source_user,target_user
0,0,PIACaraga,2021-09-21 23:53:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,"TINGNAN: Kahapon, Setyembre 20, inilabas ni @n...",TRUE,True,pcoogov,PIACaraga
1,1,PIACaraga,2021-09-21 23:53:27+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,PIACaraga
2,2,PIACaraga,2021-09-21 23:52:23+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,"PANOORIN: Kahapon, Setyembre 20, nagbigay ng u...",TRUE,True,pcoogov,PIACaraga
3,3,YamauchiElena,2021-09-21 20:45:11+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,YamauchiElena
4,4,greg_kassie,2021-09-21 16:28:08+00:00,,1.440000e+18,tl,0.0,0.0,0.0,34.0,Dapat mayroong sapat na abiso sa mga lugar na ...,MOSTLY_TRUE,True,WinGatchalian74,greg_kassie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4862,4862,roysant51162134,2021-09-17 08:29:20+00:00,,1.440000e+18,tl,0.0,0.0,1.0,0.0,@EdsonCGuido @ABSCBNNews habang dumadami nabab...,MOSTLY_FALSE,False,roysant51162134,ABSCBNNews
4863,4863,dzrhnews,2021-09-17 04:23:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,Pakiusap na payagan na rin ang mga naka-unang ...,TRUE,False,dzrhnews,liezel_once
4864,4864,GrowlHarhar,2021-09-17 03:09:24+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,@CityMouse14 @inquirerdotnet @KAguilarINQ Ikaw...,MOSTLY_TRUE,False,GrowlHarhar,inquirerdotnet
4865,4865,GrowlHarhar,2021-09-17 03:09:24+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,@CityMouse14 @inquirerdotnet @KAguilarINQ Ikaw...,MOSTLY_TRUE,False,GrowlHarhar,KAguilarINQ


In [24]:
df = df.drop(columns=[df.columns[0]])

In [25]:
filtered_dfs = []

#Sept 17
filtered_dfs.append(df.loc[(df['created_at'] >= '2021-09-16') & (df['created_at'] <= '2021-09-17')])
filtered_dfs[0] = filtered_dfs[0].reset_index(drop=True)

#Sept 18
filtered_dfs.append(df.loc[(df['created_at'] >= '2021-09-16') & (df['created_at'] <= '2021-09-18')])
filtered_dfs[1] = filtered_dfs[1].reset_index(drop=True)

#Sept 19
filtered_dfs.append(df.loc[(df['created_at'] >= '2021-09-16') & (df['created_at'] <= '2021-09-19')])
filtered_dfs[2] = filtered_dfs[2].reset_index(drop=True)

#Sept 20
filtered_dfs.append(df.loc[(df['created_at'] >= '2021-09-16') & (df['created_at'] <= '2021-09-20')])
filtered_dfs[3] = filtered_dfs[3].reset_index(drop=True)

#Sept 21
filtered_dfs.append(df.loc[(df['created_at'] >= '2021-09-16') & (df['created_at'] <= '2021-09-21')])
filtered_dfs[4] = filtered_dfs[4].reset_index(drop=True)

#Sept 22
filtered_dfs.append(df.loc[(df['created_at'] >= '2021-09-16') & (df['created_at'] <= '2021-09-22')])
filtered_dfs[5] = filtered_dfs[5].reset_index(drop=True)

In [26]:
filtered_dfs[0].to_csv('from_model_v2_17.csv')
filtered_dfs[1].to_csv('from_model_v2_18.csv')
filtered_dfs[2].to_csv('from_model_v2_19.csv')
filtered_dfs[3].to_csv('from_model_v2_20.csv')
filtered_dfs[4].to_csv('from_model_v2_21.csv')
filtered_dfs[5].to_csv('from_model_v2_22.csv')